## DIA 118: Enviar Automáticamente el PDF de Estadísticas por Correo Cada Semana

Hoy vas a automatizar el envío del PDF de estadísticas de cada usuario por correo electrónico, sin que tengan que descargarlo manualmente.

✅ ¿Qué aprenderás hoy?

🕒 Programar generación automática del PDF de estadísticas

📩 Enviar ese PDF como adjunto por email

🔄 Automatizarlo semanalmente usando schedule

🧩 1. Crear función para generar PDF de estadísticas (servidor-side)
En lugar de capturar un canvas como hacíamos en frontend, generaremos el PDF directamente en el servidor usando matplotlib + reportlab para simplicidad.

Generar datos del usuario:

python
Copiar
Editar
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import io

def generar_pdf_estadisticas(user):
    # Extraer los datos del historial
    reportes = Reporte.query.filter_by(user_id=user.id).all()

    if not reportes:
        return None  # No hay reportes aún

    # Calcular estadísticas
    total_pdf = sum(1 for r in reportes if r.tipo == 'pdf')
    total_csv = sum(1 for r in reportes if r.tipo == 'csv')
    total = len(reportes)

    pdf_buffer = io.BytesIO()
    p = canvas.Canvas(pdf_buffer, pagesize=letter)
    width, height = letter

    p.setFont("Helvetica-Bold", 16)
    p.drawString(50, height - 50, f"📊 Estadísticas de Reportes - {user.username}")

    p.setFont("Helvetica", 12)
    p.drawString(50, height - 100, f"Total de reportes: {total}")
    p.drawString(50, height - 130, f"PDF generados: {total_pdf}")
    p.drawString(50, height - 160, f"CSV generados: {total_csv}")

    p.setFont("Helvetica-Oblique", 10)
    p.drawString(50, height - 200, "Reporte generado automáticamente.")

    p.showPage()
    p.save()
    pdf_buffer.seek(0)

    return pdf_buffer
🧩 2. Función para enviar el PDF de estadísticas por correo
python
Copiar
Editar
def enviar_estadisticas_email(user, pdf_buffer):
    from email.message import EmailMessage
    import smtplib
    import os

    msg = EmailMessage()
    msg['Subject'] = f'Estadísticas Semanales - {user.username}'
    msg['From'] = os.getenv("EMAIL_SENDER")
    msg['To'] = user.email

    msg.set_content(f"""
Hola {user.username},

Adjuntamos tus estadísticas de reportes generados esta semana.

¡Gracias por usar nuestra plataforma!
""")

    msg.add_attachment(pdf_buffer.read(), maintype='application', subtype='pdf',
                       filename=f'estadisticas_{user.username}.pdf')

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.getenv("EMAIL_SENDER"), os.getenv("EMAIL_PASSWORD"))
            smtp.send_message(msg)
        print(f"✅ Estadísticas enviadas a {user.email}")
    except Exception as e:
        print(f"❌ Error al enviar estadísticas: {e}")
🧩 3. Programar el envío semanal
Agrega en tu script de tareas programadas:

python
Copiar
Editar
import schedule
import time
from app import db, app, User

def tarea_enviar_estadisticas():
    with app.app_context():
        usuarios = User.query.all()
        for user in usuarios:
            pdf_buffer = generar_pdf_estadisticas(user)
            if pdf_buffer:
                enviar_estadisticas_email(user, pdf_buffer)

schedule.every().monday.at("09:00").do(tarea_enviar_estadisticas)

print("⏳ Esperando para enviar estadísticas semanales...")

while True:
    schedule.run_pending()
    time.sleep(60)
